# Perigrine Falcon Study 

#### Falk Huettmann, University of Alaska, Fairbanks
#### Ernest Mack, Data Munging

#### This Jupyter notebook is located at: https://github.com/SeriousHyena/UAF-Temperature/blob/master/Perigrines.ipynb

### reference: http://geologyandpython.com/geology-of-MI-and-WI.html

# Import the needed libraries

In [42]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
from shapely import geometry
import matplotlib.pyplot as plt

%matplotlib inline

# Red in the Perigrines .csv file

In [43]:
Perigrines = pd.read_csv(r'C:\Users\emack\Documents\UAF-Temperature\data\pfunique2.csv') # save to a dataframe                   
Perigrines.head() # This shows the first 5 entries of the DF.


,longitude,latitude
0,48.2282,-122.3490
1,44.5620,-68.8043
2,40.7545,-73.9645
3,40.4393,-73.9869
4,41.4992,-81.7147


# Confirm the correct CRS
From the metadata we have that the geophysics data is referenced to latlong (NAD27) datum. Let's use pyproj to transform it to both latlong (WGS84) and UTM zone 16 (WGS84).

In [44]:
# North American Datum 1927
p1 = pyproj.Proj(proj='latlong', datum='NAD27')
# WGS84 Latlong
p2 = pyproj.Proj(proj='latlong', datum='WGS84')
# WGS84 UTM Zone 16
p3 = pyproj.Proj(proj='utm', zone=16, datum='WGS84')
Perigrines['long_wgs84'], Perigrines['lat_wgs84'] = pyproj.transform(p1, p2, 
                                                            Perigrines.longitude.values, 
                                                            Perigrines.latitude.values)
Perigrines['E_utm'], Perigrines['N_utm'] = pyproj.transform(p1, p3,
                                                        Perigrines.longitude.values,
                                                        Perigrines.latitude.values)

# Convert our pandas DataFrame into a geopandas GeoDataFrame

In [45]:
Perigrines['geometry'] = [geometry.Point(x, y) for x, y in zip(Perigrines['long_wgs84'], Perigrines['lat_wgs84'])]

### Pass the DataFrame to a GeoDataFrame object indicating which one is the geometry columns and the CRS (Coordinate Reference System):

In [46]:
Perigrines = gpd.GeoDataFrame(Perigrines, geometry='geometry', crs="+init=epsg:4326")

# Save the file for re-use when we start working with other layers of data

In [47]:
Perigrines.to_csv(r'C:\Users\emack\Documents\UAF-Temperature\data\mag.csv')

# Verify our new file looks ok

In [48]:
Perigrines.head(3)

,longitude,latitude,long_wgs84,lat_wgs84,E_utm,N_utm,geometry
0,48.2282,-122.3490,48.2282,-122.3490,inf,inf,POINT (48.228 -122.349)
1,44.5620,-68.8043,44.5620,-68.8043,2.273971e+06,-1.160805e+07,POINT (44.562 -68.804)
2,40.7545,-73.9645,40.7545,-73.9645,1.919624e+06,-1.111200e+07,POINT (40.755 -73.965)


### .envelope is a shapely method to get the bounding rectangle of all the points in the geometry

In [49]:
multipoints = geometry.MultiPoint(Perigrines['geometry'])
bounds = multipoints.envelope

### Let's save the bounds for the next parts of Integrating & Exploring.

In [50]:
gpd.GeoSeries(bounds).to_file(r'C:\Users\emack\Documents\UAF-Temperature\data\area_of_study_bounds.gpkg', 'GPKG')

### Let's get the coordinates of the .boundary of the bounds.

In [51]:
coords = np.vstack(bounds.boundary.coords.xy)

### Calculate the mean of the coordinates using numpy to center the location map on this.

In [52]:
UAF_coords = (64.8558, -147.8335)
map_center = list(coords.mean(1))[::-1] # use a slice to get all the rows from the last column 

In [53]:
UAF_coords

(64.8558, -147.8335)

# Anecdotaly zoom into Fairbanks, Alaska

In [54]:
m = folium.Map(location=UAF_coords, zoom_start=8, control_scale=True)
folium.PolyLine(coords[::-1].T).add_to(m)
folium.LatLngPopup().add_to(m)
m

# Display the first 50 rows

The 'long_wgs84 and lat_wgs84' columns confirm we have the correct CRS as they are the same as initial read from the .csv. The 'geometry' column is what we will use for later bounding-box zooming. the E_utm and N_utm are described here: https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system

In [55]:
Perigrines.head(50)

,longitude,latitude,long_wgs84,lat_wgs84,E_utm,N_utm,geometry
0,48.22820,-122.34900,48.22820,-122.34900,inf,inf,POINT (48.228 -122.349)
1,44.56200,-68.80430,44.56200,-68.80430,2.273971e+06,-1.160805e+07,POINT (44.562 -68.804)
2,40.75450,-73.96450,40.75450,-73.96450,1.919624e+06,-1.111200e+07,POINT (40.755 -73.965)
3,40.43930,-73.98690,40.43930,-73.98690,1.923848e+06,-1.110263e+07,POINT (40.439 -73.987)
4,41.49920,-81.71470,41.49920,-81.71470,1.224469e+06,-1.057624e+07,POINT (41.499 -81.715)
5,44.43410,-73.25140,44.43410,-73.25140,1.903800e+06,-1.125511e+07,POINT (44.434 -73.251)
6,36.53490,-121.87800,36.53490,-121.87800,inf,inf,POINT (36.535 -121.878)
7,42.45500,-122.95000,42.45500,-122.95000,inf,inf,POINT (42.455 -122.950)
8,41.90630,-87.70200,41.90630,-87.70200,6.996664e+05,-1.015915e+07,POINT (41.906 -87.702)
9,-2.48677,-79.61160,-2.48677,-79.61160,1.660674e+06,-9.885857e+06,POINT (-2.487 -79.612)
